# Visualize Decoded Results

In [6]:
import math
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
from scipy import spatial

from nuscenes.nuscenes import NuScenes
from pyquaternion.quaternion import Quaternion
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix
from nuscenes.utils.data_classes import PointCloud, LidarPointCloud, RadarPointCloud, Box
from matplotlib import cm
from open3d import JVisualizer

import torch
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from tqdm.auto import tqdm

from data_splitter import DataSplitter
from training_set import TrainingSetLidarSeg
from loss import *
from model import Model
from sphere import Sphere
from visualize import Visualize
from metrics import *
from average_meter import AverageMeter
    
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
print(f"Initializing CUDA...")
torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True

print(f"Setting parameters...")
bandwidth = 100
batch_size = 1
num_workers = 32
n_classes = 9
device_ids = [0]

print(f"Initializing data structures...")
criterion = MainLoss()

writer = SummaryWriter()
# stored_model = 'test_lidarseg_20220502120200.pkl'
stored_model = './test_lidarseg_20220502120200.pkl'
model = Model(bandwidth=bandwidth, n_classes=n_classes).cuda()
net = nn.DataParallel(model, device_ids = device_ids).to(0)

net.load_state_dict(torch.load(stored_model))

# chkp = './checkpoints/test_lidarseg_20220502120200_8.pth'
# checkpoint = torch.load(chkp)
# net.load_state_dict(checkpoint['model_state_dict'])


print(f"All instances initialized.")

Initializing CUDA...
Setting parameters...
Initializing data structures...
[Model] We have [2, 20, 40, 120, 180, 120, 40, 20, 9] features.
[Model] We have [100, 40, 30, 15, 10, 8, 10, 15, 30, 40, 100] bandwidths.
All instances initialized.


In [7]:
# export_ds = '/mnt/data/datasets/nuscenes/processed'
export_ds = '/media/scratch/berlukas/nuscenes'
cloud_filename = f"{export_ds}/sem_clouds_val_new.npy"
dec_clouds = f"{export_ds}/sem_clouds_val_decoded.npy"
dec_gt_clouds = f"{export_ds}/sem_clouds_gt.npy"

print(f"Loading clouds from {cloud_filename}.")
cloud_features = np.load(cloud_filename)

sem_cloud_features = cloud_features[:, 2, :, :]
cloud_features = cloud_features[:, 0:2, :, :]
print(f"Shape of clouds is {cloud_features.shape} and sem clouds is {sem_cloud_features.shape}")

# Initialize the data loaders
train_set = TrainingSetLidarSeg(cloud_features, sem_cloud_features)
print(f"Total size of the training set: {len(train_set)}")
split = DataSplitter(train_set, True, test_train_split=0.0, val_train_split=0.0, shuffle=False)

# Split the data into train, val and optionally test
_, _, data_loader = split.get_split(
    batch_size=batch_size, num_workers=num_workers)
data_size = split.get_test_size()

print("Dataset size for testing: ", data_size)

Loading clouds from /media/scratch/berlukas/nuscenes/sem_clouds_val_new.npy.
Shape of clouds is (6053, 2, 200, 200) and sem clouds is (6053, 200, 200)
Total size of the training set: 6053
Dataset size for testing:  6053


In [8]:
def test_lidarseg(net, criterion, writer):
    all_decoded_clouds = [None] * data_size
    all_gt_clouds = [None] * data_size
    k = 0
    avg_pixel_acc = AverageMeter()
    avg_pixel_acc_per_class = AverageMeter()
    avg_jacc = AverageMeter()
    avg_dice = AverageMeter()
    n_iter = 0
    net.eval()
    with torch.no_grad():            
        for batch_idx, (cloud, lidarseg_gt) in enumerate(tqdm(data_loader)):
            cloud, lidarseg_gt = cloud.cuda().float(), lidarseg_gt.cuda().long()
            enc_dec_cloud = net(cloud)
            
            pred_segmentation = torch.argmax(enc_dec_cloud, dim=1)
            pixel_acc, pixel_acc_per_class, jacc, dice = eval_metrics(lidarseg_gt, pred_segmentation, num_classes = n_classes)
            avg_pixel_acc.update(pixel_acc)
            avg_pixel_acc_per_class.update(pixel_acc_per_class)
            avg_jacc.update(jacc)
            avg_dice.update(dice)
            
            writer.add_scalar('Test/PixelAccuracy', pixel_acc, n_iter)   
            writer.add_scalar('Test/PixelAccuracyPerClass', pixel_acc_per_class, n_iter)   
            writer.add_scalar('Test/JaccardIndex', jacc, n_iter)
            writer.add_scalar('Test/DiceCoefficient', dice, n_iter)  
            
            n_batch = enc_dec_cloud.shape[0]
            for i in range(0, n_batch):                
                all_decoded_clouds[k] = enc_dec_cloud.cpu().data.numpy()[i,:,:,:]
                all_gt_clouds[k] = lidarseg_gt.cpu().data.numpy()[i,:,:]
                k = k + 1     
            n_iter += 1
            
        writer.add_scalar('Test/AvgPixelAccuracy', avg_pixel_acc.avg, n_iter)   
        writer.add_scalar('Test/AvgPixelAccuracyPerClass', avg_pixel_acc_per_class.avg, n_iter)   
        writer.add_scalar('Test/AvgJaccardIndex', avg_jacc.avg, n_iter)
        writer.add_scalar('Test/AvgDiceCoefficient', avg_dice.avg, n_iter)  
        
        print(f'Average Pixel Accuracy: {avg_pixel_acc.avg}')
        print(f'Average Pixel Accuracy per Class: {avg_pixel_acc_per_class.avg}')
        print(f'Average Jaccard Index: {avg_jacc.avg}')
        print(f'Average DICE Coefficient: {avg_dice.avg}')

    return np.array(all_decoded_clouds), np.array(all_gt_clouds)

print("Starting testing...")

torch.cuda.empty_cache()
decoded_clouds, gt_clouds = test_lidarseg(net, criterion, writer)
print(f'Decoded clouds are {decoded_clouds.shape}')
print(f'Decoded gt clouds are {gt_clouds.shape}')

# np.save(dec_gt_clouds, gt_clouds)
np.save(dec_clouds, decoded_clouds)
print(f'Wrote decoded spheres to {dec_clouds}.')

writer.close()
print("Testing finished!")

Starting testing...



Average Pixel Accuracy: 0.9613656401634216
Average Pixel Accuracy per Class: 0.6270266771316528
Average Jaccard Index: 0.5282055139541626
Average DICE Coefficient: 0.5994076728820801
Decoded clouds are (6053, 9, 200, 200)
Decoded gt clouds are (6053, 200, 200)
Wrote decoded spheres to /media/scratch/berlukas/nuscenes/sem_clouds_val_decoded.npy.
Testing finished!


In [33]:
def CreateGrid(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([2, n_grid, n_grid])
    for i in range(n_grid):
        for j in range(n_grid):
            points[0, i, j] = (np.pi*(2*i+1))/(4*bw)
            points[1, i, j] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points

def createGrid_old(bw):
        n_grid = 2 * bw
        k = 0;
        points = np.empty([n_grid * n_grid, 2])
        for i in range(n_grid):
            for j in range(n_grid):
                points[k, 0] = (np.pi*(2*i+1))/(4*bw)
                points[k, 1] = (2*np.pi*j)/(2*bw);
                k = k + 1;
        return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])    
    return cart_grid

def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

class SamplingPointCloud(PointCloud):

    @staticmethod
    def nbr_dims() -> int:
        return 4
    
    @classmethod
    def from_file(cls, file_name: str) -> 'SamplingPointCloud':
        return None
    
    @classmethod
    def from_bw(cls, bw, scale = 100) -> 'SamplingPointCloud':
        grid = createGrid_old(bw)
        xyz_grid = convertGridToEuclidean_old(grid) * scale
        intensities = np.zeros((xyz_grid.shape[0],1))
        sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
        return cls(sampling_grid.T)

def create_spherical_pc(feature, trans = 0, bw = 100):
    pc = SamplingPointCloud.from_bw(bw, 1)   
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_xyz[:, 0:3])
    colors = mapIntensityToRGB(feature[:, 0])
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd    

def create_cloud_pc(cloud, trans = 0):
    cloud[:,0] = cloud[:,0] + trans
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    return pcd
  
def compare_estimation_clouds(decoded, gt, bw = 100):  
    decoded_pc = create_cloud_pc(decoded, trans=0)
    gt_pc = create_cloud_pc(gt, trans=100)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])
    
def backproject_cloud(spherical, distance, bw = 100):    
    grid, _ = DHGrid.CreateGrid(bw)
    n_points = grid.shape[1] * grid.shape[2]
    cart_sphere = np.zeros([n_points, 4])
    k = 0
    for i in range(0, grid.shape[1]):
        for j in range(0, grid.shape[2]):
            dist = distance[i,j]
            if dist <= 0:
                continue
            cart_sphere[k,0] = dist * np.multiply(np.cos(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,1] = dist * np.multiply(np.sin(grid[1,i,j]), np.sin(grid[0,i,j]))
            cart_sphere[k,2] = dist * np.cos(grid[0,i,j])
            cart_sphere[k,3] = spherical[i,j]
            k = k + 1
    return cart_sphere  

def prepare_for_viz(cloud):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(cloud[:, 0:3])
    if cloud.shape[1] == 4:
        colors = mapIntensityToRGB(cloud[:, 3])
        pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    if cloud.shape[1] == 6:
        pcd.colors = o3d.utility.Vector3dVector(cloud[:,3:6] / 255.0)
    return pcd

def convert_sphere(sph, feature_idx, n_features, bw, trans = 0.0):
    sph = np.reshape(sph, (n_features, -1)).T
    
    pc = SamplingPointCloud.from_bw(bw, 1)
    points_xyz = pc.points.T[:,0:3]
    points_xyz[:,0] = points_xyz[:,0] + trans
    return np.column_stack((points_xyz, sph[:,feature_idx]))
    
def visualize_pointcloud(cloud):
    pcd = prepare_for_viz(cloud)
    o3d.visualization.draw_geometries([pcd])

def visualize_sphere(sph, feature_idx = 0, n_features = 3, bw = 100):
    points_xyzi = convert_sphere(sph, feature_idx, n_features, bw)
    visualize_pointcloud(points_xyzi)
    
def compare_estimation_sphere(decoded, gt, bw = 100):  
    decoded_pc = create_spherical_pc(decoded, trans=0, bw=bw)
    gt_pc = create_spherical_pc(gt, trans=2.5, bw=bw)
    o3d.visualization.draw_geometries([decoded_pc, gt_pc])    
    
def compare_estimation_sphere2(decoded, gt, bw = 100):  
    decoded_pc = convert_sphere(decoded, 0, 1, 100)
    gt_pc = convert_sphere(gt, 2, 3, 100, 2.5)
    
    decoded_pcd = prepare_for_viz(decoded_pc)
    gt_pcd = prepare_for_viz(gt_pc)
    o3d.visualization.draw_geometries([decoded_pcd, gt_pcd]) 

def compare_estimation_sphere3(decoded, gt, bw = 100):  
    decoded_pcd = create_spherical_pc(decoded, trans=0, bw=bw)
    gt_pcd = create_spherical_pc(gt, trans=2.5, bw=bw)
    
    visualizer = JVisualizer()
    visualizer.add_geometry(decoded_pcd)
    visualizer.show()
    

In [36]:
n_decoded = decoded_clouds.shape[0]

avg_pixel_acc = AverageMeter()
avg_pixel_acc_per_class = AverageMeter()
avg_jacc = AverageMeter()
avg_dice = AverageMeter()
for i in range(15, 16):
    cur_decoded = decoded_clouds[i, :, :, :]
    cur_decoded = np.argmax(cur_decoded, axis=0)
    cur_sem_cloud = gt_clouds[i, :, :]
    cur_input = cloud_features[i, :, :]
    
    pred_segmentation = torch.from_numpy(cur_decoded).cuda().int()
    gt_segmentation = torch.from_numpy(cur_sem_cloud).cuda().int()
    pixel_acc, pixel_acc_per_class, jacc, dice = eval_metrics(gt_segmentation, pred_segmentation, num_classes = 9)
    avg_pixel_acc.update(pixel_acc)
    avg_pixel_acc_per_class.update(pixel_acc_per_class)
    avg_jacc.update(jacc)
    avg_dice.update(dice)
        
    cur_decoded = np.reshape(cur_decoded, (1, -1)).T    
    cur_sem_cloud = np.reshape(cur_sem_cloud, (1, -1)).T    
    
    compare_estimation_sphere3(cur_decoded, cur_sem_cloud, 100)

print(f'overall pixel acc = {avg_pixel_acc.avg}')
print(f'pixel acc per class = {avg_pixel_acc_per_class.avg}')
print(f'mean jaccard index = {avg_jacc.avg}')
print(f'mean dice coeff = {avg_dice.avg}')

JVisualizer with 1 geometries

overall pixel acc = 0.9174749851226807
pixel acc per class = 0.3855117857456207
mean jaccard index = 0.3682934045791626
mean dice coeff = 0.39813151955604553


In [14]:
i = 1

cur_decoded = decoded_clouds[i, :, :, :]
cur_decoded = np.argmax(cur_decoded, axis=0)

# cur_cloud = decoded_gt[i, :, :, :]
# compare_estimation_sphere2(cur_decoded, cur_cloud)

# compare_estimation_sphere(cur_decoded, cur_cloud)
visualize_sphere(cur_decoded, 0, 1, 125)
# visualize_sphere(cur_cloud, 2, 3, 100)
# est_cloud = backproject_cloud(cur_decoded, cur_input[0,:,:], bw)
# gt_cloud = backproject_cloud(cur_sem_sphere, cur_input[0,:,:], bw)